## Imports

In [1]:
import xarray as xr
import numpy as np
import imageio
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib.colors import Normalize
import datetime
from pathlib import Path

## Setup

In [26]:
nc_path = "/N/slate/jmelms/projects/FCN_dynamical_testing/data/output/test2.nc"
lat_path = "/N/u/jmelms/BigRed200/projects/dynamical-tests-FCN/metadata/latitude.npy"
lon_path = "/N/u/jmelms/BigRed200/projects/dynamical-tests-FCN/metadata/longitude.npy"
lsm_path = "/N/u/jmelms/BigRed200/projects/dynamical-tests-FCN/metadata/land_sea_edges_mask.npy"
mean_path = "/N/u/jmelms/BigRed200/projects/dynamical-tests-FCN/metadata/global_means.npy"
std_path = "/N/u/jmelms/BigRed200/projects/dynamical-tests-FCN/metadata/global_stds.npy"
img_out_path = Path("/N/u/jmelms/BigRed200/projects/dynamical-tests-FCN/code/analysis/imgs")
img_out_path.mkdir(exist_ok=True)

## Loading Data

In [28]:
# load lat, lon, and land-sea mask
lat = np.load(lat_path)
lon = np.load(lon_path)
lsm = np.load(lsm_path)
mean = np.load(mean_path)
std = np.load(std_path)
nlat = lat.size
nlon = lon.size
dt = 6 # time step in hours

# load netcdf data
ds = xr.open_dataset(nc_path)

# get the data array
da = ds["__xarray_dataarray_variable__"]

# remove the "history" dimension because it's not used in these runs
da = da.squeeze(drop=True)

# check to make sure that the lat/lon dimensions are the same as the input data
assert da.lat.size == nlat, "Latitude dimensions do not match"
assert da.lon.size == nlon, "Longitude dimensions do not match"

# change time dimension to datetime.datetime if using real times
# ds["time"] = [datetime.datetime.fromtimestamp(t/10e8, tz=datetime.timezone.utc) for t in ds.time.values]

# change time dimension to hours from initialization if using relative times for idealized sim
da["time"] = [t/10e8/3600 for t in da.time.values]

# unstandardize data
da = da * std + mean

# flatten mean and stds
mean = mean.squeeze()
std = std.squeeze()

print(da)

<xarray.DataArray '__xarray_dataarray_variable__' (time: 4, channel: 73,
                                                   lat: 721, lon: 1440)>
array([[[[ 7.45058060e-09,  7.45058060e-09,  7.45058060e-09, ...,
           7.45058060e-09,  7.45058060e-09,  7.45058060e-09],
         [ 7.45058060e-09,  7.45058060e-09,  7.45058060e-09, ...,
           7.45058060e-09,  7.45058060e-09,  7.45058060e-09],
         [ 7.45058060e-09,  7.45058060e-09,  7.45058060e-09, ...,
           7.45058060e-09,  7.45058060e-09,  7.45058060e-09],
         ...,
         [ 7.45058060e-09,  7.45058060e-09,  7.45058060e-09, ...,
           7.45058060e-09,  7.45058060e-09,  7.45058060e-09],
         [ 7.45058060e-09,  7.45058060e-09,  7.45058060e-09, ...,
           7.45058060e-09,  7.45058060e-09,  7.45058060e-09],
         [ 7.45058060e-09,  7.45058060e-09,  7.45058060e-09, ...,
           7.45058060e-09,  7.45058060e-09,  7.45058060e-09]],

        [[-4.47034836e-08, -4.47034836e-08, -4.47034836e-08, ...,
    

## Check a few timesteps of mslp

In [35]:
def context():
    vis_name = "mslp_time_series"
    output_dir = img_out_path / vis_name
    output_dir.mkdir(exist_ok=True, parents=True)

    channel = "mslp"
    units = "Pa"
    xticks = np.arange(0, nlon, 200)
    yticks = np.arange(0, nlat, 200)
    xticklabs = lon[xticks]
    yticklabs = lat[yticks]
    contour_levels = np.arange(900, 1100, 4) * 100

    # hPa = 100 Pa
    data = da.sel(channel="msl")

    vmin = data.min()
    vmax = data.max()

    # vmin = 800
    # vmax = 1200

    tstep = data.isel(time=1)
    
    # plot all timesteps
    for i, t in enumerate(da.time):
        if i % 10:
            continue
        t_str = str(t.values)
        tstep = data.sel(time=t)
        fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(10, 5), gridspec_kw={'width_ratios': [1, 0.05]})
        ax1.imshow(tstep, cmap='viridis')
        CS = ax1.contour(tstep, levels=3, colors="black")
        ax1.clabel(CS, CS.levels, inline=True, fmt=lambda x: round(x), fontsize=10)
        norm = Normalize(vmin=vmin, vmax=vmax)
        cmap = cm.viridis
        scalar_mappable = cm.ScalarMappable(norm=norm, cmap=cmap)
        fig.colorbar(scalar_mappable, label=f"{channel} ({units})", cax=ax2, fraction=0.05, pad=0.07, )
        fig.suptitle(f'{channel} ({units}) at {t_str}h')
        ax1.set_xlabel('Longitude')
        ax1.set_ylabel('Latitude')
        ax1.set_xticks(xticks, xticklabs)
        ax1.set_yticks(yticks, yticklabs)

        fig.savefig(output_dir / f"{vis_name}_{i:04d}.png")
        print(f"Saved {output_dir / f'{vis_name}_{i:04d}.png'}")
        plt.close()

context()

Saved /N/u/jmelms/BigRed200/projects/dynamical-tests-FCN/code/analysis/imgs/mslp_time_series/mslp_time_series_0000.png
Saved /N/u/jmelms/BigRed200/projects/dynamical-tests-FCN/code/analysis/imgs/mslp_time_series/mslp_time_series_0010.png
Saved /N/u/jmelms/BigRed200/projects/dynamical-tests-FCN/code/analysis/imgs/mslp_time_series/mslp_time_series_0020.png
Saved /N/u/jmelms/BigRed200/projects/dynamical-tests-FCN/code/analysis/imgs/mslp_time_series/mslp_time_series_0030.png
Saved /N/u/jmelms/BigRed200/projects/dynamical-tests-FCN/code/analysis/imgs/mslp_time_series/mslp_time_series_0040.png
Saved /N/u/jmelms/BigRed200/projects/dynamical-tests-FCN/code/analysis/imgs/mslp_time_series/mslp_time_series_0050.png


### Checking all vars

In [48]:
def context():
    vis_name = "channel_inits"
    output_dir = img_out_path / vis_name
    output_dir.mkdir(exist_ok=True, parents=True)

    time = 0
    xticks = np.arange(0, nlon, 200)
    yticks = np.arange(0, nlat, 200)
    xticklabs = lon[xticks]
    yticklabs = lat[yticks]
    # contour_levels = np.arange(900, 1100, 4)

    data = da.sel(time=time)
    
    # plot all timesteps
    for i, c in enumerate(da.channel):
        c_str = str(c.values)
        if c_str != "msl":
            continue
        tstep = data.isel(channel=i)
        vmin = tstep.min().values
        vmax = tstep.max().values
        print(f"-/+ {c_str}: {vmin:.3f}/{vmax:.3f}")
        fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(10, 5), gridspec_kw={'width_ratios': [1, 0.05]})
        ax1.imshow(tstep, cmap='viridis', vmin=vmin, vmax=vmax)
        CS = ax1.contour(tstep, levels=3, colors="black")
        ax1.clabel(CS, CS.levels, inline=True, fmt=lambda x: round(x), fontsize=10)
        norm = Normalize(vmin=vmin, vmax=vmax)
        cmap = cm.viridis
        scalar_mappable = cm.ScalarMappable(norm=norm, cmap=cmap)
        fig.colorbar(scalar_mappable, label=f"{c_str}", cax=ax2, fraction=0.05, pad=0.07, )
        fig.suptitle(f'{c_str} at 0h')
        ax1.set_xlabel('Longitude')
        ax1.set_ylabel('Latitude')
        ax1.set_xticks(xticks, xticklabs)
        ax1.set_yticks(yticks, yticklabs)

        fig.savefig(output_dir / f"{vis_name}_{i:04d}.png")
        # print(f"Saved {output_dir / f'{vis_name}_{i:04d}.png'}")
        print(tstep)
        plt.close()

context()

-/+ msl: 101330.562/101330.562
<xarray.DataArray '__xarray_dataarray_variable__' (lat: 721, lon: 1440)>
array([[101330.56, 101330.56, 101330.56, ..., 101330.56, 101330.56,
        101330.56],
       [101330.56, 101330.56, 101330.56, ..., 101330.56, 101330.56,
        101330.56],
       [101330.56, 101330.56, 101330.56, ..., 101330.56, 101330.56,
        101330.56],
       ...,
       [101330.56, 101330.56, 101330.56, ..., 101330.56, 101330.56,
        101330.56],
       [101330.56, 101330.56, 101330.56, ..., 101330.56, 101330.56,
        101330.56],
       [101330.56, 101330.56, 101330.56, ..., 101330.56, 101330.56,
        101330.56]], dtype=float32)
Coordinates:
  * lat      (lat) float64 90.0 89.75 89.5 89.25 ... -89.25 -89.5 -89.75 -90.0
  * lon      (lon) float64 0.0 0.25 0.5 0.75 1.0 ... 359.0 359.2 359.5 359.8
    channel  <U5 'msl'
    time     float64 0.0


In [36]:
da.sel(channel="msl").isel(time=slice(0,3)).min(dim=["lat", "lon"])

<xarray.DataArray '__xarray_dataarray_variable__' (time: 3)>
array([1.0133056e+05, 9.0678424e+07, 1.0381826e+08], dtype=float32)
Coordinates:
    channel  <U5 'msl'
  * time     (time) float64 0.0 6.0 12.0

In [30]:
for channel, m, v in zip(da.channel.values, mean, da.isel(time=0, lat=0, lon=0).values):
    print(f"{channel}, {m}, {v:.7f}")

# channels = da.channel.values
# mean.flatten()

u10m, -0.07404755800962448, 0.0000000
v10m, 0.19169427454471588, -0.0000000
u100m, -0.014596237801015377, -0.0000000
v100m, 0.1996840387582779, 0.0000001
t2m, 278.6412048339844, 260.0200806
sp, 96659.890625, 101302.0703125
msl, 100966.71875, 101330.5625000
tcwv, 18.4080867767334, 3.4581280
u50, 5.533691883087158, -0.0000033
u100, 10.243287086486816, 0.0000000
u150, 13.465484619140625, 0.0000057
u200, 14.088746070861816, -0.0000067
u250, 13.219594955444336, -0.0000010
u300, 11.678762435913086, 0.0000038
u400, 8.723402976989746, -0.0000057
u500, 6.4944610595703125, 0.0000048
u600, 4.757308483123779, 0.0000024
u700, 3.2801318168640137, 0.0000033
u850, 1.3569166660308838, 0.0000002
u925, 0.5785491466522217, 0.0000001
u1000, -0.05946159362792969, -0.0000000
v50, 0.003080817172303796, 0.0000000
v100, 0.0011228377697989345, -0.0000000
v150, -0.049639418721199036, -0.0000000
v200, -0.046308573335409164, -0.0000000
v250, -0.03826545551419258, 0.0000000
v300, -0.030652925372123718, -0.0000000
v4

In [32]:
msls = ds.sel(channel="msl")["__xarray_dataarray_variable__"].isel(time=slice(0,3), lat=0, lon=0).values
mslus = msls * std[6] + mean[6]
mslus

array([[1.0133056e+05],
       [9.6606008e+07],
       [1.0779178e+08]], dtype=float32)

In [31]:
ex = 0.26925582
exus = ex * std[6] + mean[6]
exus

101325.00000354329

In [33]:
ncin = xr.open_dataset(nc_path)["__xarray_dataarray_variable__"]
ncin.sel(channel="msl").isel(time=0, lat=0, lon=0).values

array([0.2734375], dtype=float32)